# Betaseries Pull BBx


In [1]:
import glob
import os
import pandas as pd
from os import listdir
from shutil import rmtree
from subprocess import check_output
import argparse
import subprocess
import re

In [2]:
### Helper Functions 


# feat1 fsf file creation 
def make_file(sub_id, ses_id, trial_id, output_dir, task,data_dict, design_file, newkey, run):
    
    with open(design_file,'r') as infile:
        tempfsf=infile.read()
        #
        if not os.path.exists(os.path.join(output_dir, "design_files")):
            os.makedirs(os.path.join(output_dir, "design_files"))
        #print(output_dir)
        
        
        task_name=trial_id.replace(sub_id+'_',"")
        design_fileout = os.path.join(output_dir, "design_files/%s_%s_%s_beta1.fsf"%(sub_id, ses_id, task_name))
        out_param = data_dict[sub_id][newkey]["TRIALS"]["TRIAL%s"%trial_id]["OUTPUT"]
        func_param = data_dict[sub_id][newkey]["FUNCRUN"]
        confound_param = data_dict[sub_id][newkey]["CONFOUND"]
        trial_param = data_dict[sub_id][newkey]["TRIALS"]["TRIAL%s"%trial_id]["TRIAL"]
        nuis_param = data_dict[sub_id][newkey]["TRIALS"]["TRIAL%s"%trial_id]["NUIS"]
        #print(design_fileout)
        tempfsf = tempfsf.replace("OUTPUT", out_param)
        tempfsf = tempfsf.replace("FUNCRUN", func_param) 
        tempfsf = tempfsf.replace("CONFOUND", confound_param)
        tempfsf = tempfsf.replace("TRIAL", trial_param)
        tempfsf = tempfsf.replace("NUIS", nuis_param)

        for i in range(6):
            moco = data_dict[sub_id][newkey]["MOCO%i"%i]
            tempfsf = tempfsf.replace("MOCO%i"%i, moco)
        #print(tempfsf)
        try:
            with open(design_fileout,'w') as outfile:
                outfile.write(tempfsf)
            outfile.close()
        except:
            print("BAD SUBJECT ", sub_id)
        infile.close()
        
        
def create_fsf(input_dir, deriv_dir, ses_id, design_file, ev_path):
    
    ses_id=ses_id
    preproc_folder = deriv_dir
    
    data_dict={}
    
    
    # start loop -- looping through subjects
    subject_list = glob.glob(os.path.join(preproc_folder, 'sub-*/%s'%ses_id))
    for sub_path in subject_list:
        # set variables from file path
        sub_id=sub_path.split("/")[-2]
        # add subject id to dictionary
        if sub_id not in data_dict:
            data_dict[sub_id] = {}
            
            
            
        # get functional task files 
        # this section may be unique to an individual study,
        # make sure the extraction applies correctly. 
        
        # get list of nifti functional files of current subject
        # we use the `_preproc_bold_brain.nii.gz task files, these are preprocessed niftis
        functional_tasks = glob.glob(os.path.join(sub_path, 'func/*preproc_bold_brain.nii.gz'))
                
        
        for functional in functional_tasks: 
            # set specific variables from filename
            task=functional.split("/")[-1].split("_")[2].split("-")[1]
            if 'resting' in task:
                run=""
                pass
            else:
                run=functional.split("/")[-1].split("_")[3]

                
                #print(task,run)

                analysis_folder=os.path.join(deriv_dir, '%s/%s'%(sub_id,ses_id), "analysis")
                #print(analysis_folder)
                output_dir = os.path.join(analysis_folder, 'beta/task-%s_%s'%(task,run))
                #print(output_dir)
                if not os.path.exists(output_dir):
                    os.makedirs(output_dir)

                # set confound file
                #sub-001_ses-1_task-training_run-1_space-MNI152NLin2009cAsym_desc-preproc_confound.txt

                if 'resting' in task:
                    confound_file = os.path.join(sub_path, "func/motion_assessment/%s_%s_task-%s_space-MNI152NLin2009cAsym_desc-preproc_confound.txt"%(sub_id,ses_id, task))
                else:
                    confound_file = os.path.join(sub_path, "func/motion_assessment/%s_%s_task-%s_%s_space-MNI152NLin2009cAsym_desc-preproc_confound.txt"%(sub_id,ses_id, task, run))

                #print(confound_file)
                #print(run)
                if 'run' in run:
                    newkey="%s_%s"%(task, run)
                else:
                    newkey="%s"%task
                #print(newkey)
                data_dict[sub_id][newkey] = {
                    "TRIALS" : { },
                    "CONFOUND" : confound_file,
                    "FUNCRUN" : functional
                  }           

                # set motion parameters
                # sub-001_ses-1_task-training_run-1_moco0.txt | sub-001_ses-1_task-resting_moco5.txt
                for i in range(6):
                    if 'run' in run:
                        motcor=os.path.join(sub_path, 'func','motion_assessment', 'motion_parameters','%s_%s_task-%s_%s_moco%s.txt'%(sub_id,ses_id, task,run,i))
                    else:
                        motcor=os.path.join(sub_path, 'func','motion_assessment', 'motion_parameters','%s_%s_task-%s_moco%s.txt'%(sub_id,ses_id, task,i))
                    #print(motcor)
                    data_dict[sub_id][newkey]['MOCO%i'%i] = motcor

                #print(data_dict)

                # setup evs(onsets)
                trial_evs = sorted(glob.glob(os.path.join(ev_path, '%s_*%s*.txt'%(sub_id, run))))
                #print(trial_evs[0])

                if not trial_evs:
                    pass
                else:
                    for trial_file in trial_evs:
                        _id = sub_id.split("-")[1]
                        _id = _id[1:]
                        trial_id = trial_file.split("/")[-1].split(".")[0]
                        #print(trial_id)
                        nuis_file = os.path.join(ev_path, '%s.txt'%trial_id.replace('trial', 'nuis'))
                        #print(nuis_file)
                        
                        trial_ext=trial_id.replace(sub_id+'_',"")
                        fileout = os.path.join(output_dir, "%s_%s_%s"%(sub_id, ses_id,trial_ext))
                        #print(fileout)
                        
                        # fill dictionary
                        data_dict[sub_id][newkey]["TRIALS"]["TRIAL%s"%trial_id] = {"TRIAL" : trial_file, "NUIS": nuis_file, "OUTPUT" : fileout}
                        
                        # write out file
                        make_file(sub_id, ses_id, trial_id,output_dir, task, data_dict,design_file, newkey, run)

                
    return(data_dict) 
        

In [3]:
def set_paths():
    #global input_dir
    #global deriv_dir
   
    return input_dir, deriv_dir,design_file



In [4]:
def main():
    ses_id='ses-1'
    input_dir = '/projects/niblab/experiments/bbx/data'    
    deriv_dir= os.path.join(input_dir, 'bids/derivatives/preprocessed')
    design_file='/projects/niblab/design_file_templates/beta_design.fsf'
    ev_path='/projects/niblab/experiments/bbx/data/onsets/trials'
    print('[INFO] making fsf design files...')
    #data_dict=create_fsf(input_dir, deriv_dir,ses_id,design_file,ev_path)
    print('[INFO] process complete.')

    
main()


[INFO] making fsf design files...
[INFO] process complete.


Lets view a few of files created

In [5]:
# look at created files
output_path='/projects/niblab/experiments/bbx/data/bids/derivatives/preprocessed/sub-001/ses-1/analysis/beta/task-training_run-4/design_files'
listdir(output_path)[:5]
        

['sub-001_ses-1_task-H2O-run-4_nuis1_beta1.fsf',
 'sub-001_ses-1_task-H2O-run-4_nuis2_beta1.fsf',
 'sub-001_ses-1_task-H2O-run-4_nuis3_beta1.fsf',
 'sub-001_ses-1_task-H2O-run-4_nuis4_beta1.fsf',
 'sub-001_ses-1_task-H2O-run-4_nuis5_beta1.fsf']

In [6]:
beta1_files_path='/projects/niblab/experiments/bbx/data/bids/derivatives/preprocessed/sub-*/ses-1/analysis/beta/task-training_run-*/design_files/*.fsf'
print('[INFO] %f fsf files available.'%len(glob.glob(beta1_files_path)))


[INFO] 64784.000000 fsf files available.


In [7]:
# view a few of the files
glob.glob(beta1_files_path)[:5]

['/projects/niblab/experiments/bbx/data/bids/derivatives/preprocessed/sub-001/ses-1/analysis/beta/task-training_run-4/design_files/sub-001_ses-1_task-H2O-run-4_nuis1_beta1.fsf',
 '/projects/niblab/experiments/bbx/data/bids/derivatives/preprocessed/sub-001/ses-1/analysis/beta/task-training_run-4/design_files/sub-001_ses-1_task-H2O-run-4_nuis2_beta1.fsf',
 '/projects/niblab/experiments/bbx/data/bids/derivatives/preprocessed/sub-001/ses-1/analysis/beta/task-training_run-4/design_files/sub-001_ses-1_task-H2O-run-4_nuis3_beta1.fsf',
 '/projects/niblab/experiments/bbx/data/bids/derivatives/preprocessed/sub-001/ses-1/analysis/beta/task-training_run-4/design_files/sub-001_ses-1_task-H2O-run-4_nuis4_beta1.fsf',
 '/projects/niblab/experiments/bbx/data/bids/derivatives/preprocessed/sub-001/ses-1/analysis/beta/task-training_run-4/design_files/sub-001_ses-1_task-H2O-run-4_nuis5_beta1.fsf']

## Submit slurm jobs

Tasks Available

In [8]:
tasks=[x.split("/")[-1].split("_")[2].split("-")[1] for x in glob.glob(output_path+"/*.fsf")]
set(tasks)


{'H2O', 'H2Ocue', 'SSB', 'SSBcue', 'USB', 'USBcue', 'rinse'}

In [20]:
def by_file_slurm(subject):
    fsf_path='/projects/niblab/experiments/bbx/data/bids/derivatives/preprocessed/{}/ses-1/analysis/beta/task-training_run-*/design_files/*SSB*.fsf'.format(subject)
    fsfs=glob.glob(fsf_path)
    for fsf in fsfs:
        slurm_cmd = "sbatch /projects/niblab/experiments/bbx/data/code/beta_by_file.job {}".format(fsf)
        #print('[INFO] submitted: \n', slurm_cmd)
        #os.system(slurm_cmd)
    


In [21]:
# get subject ids
subject_ids=[x.split('/')[-2] for x in glob.glob('/projects/niblab/experiments/bbx/data/bids/derivatives/preprocessed/sub-*/ses-1')]



In [25]:
subject_ids[:5] # view first 5 subject ids

['sub-001', 'sub-002', 'sub-003', 'sub-004', 'sub-005']

In [26]:
# run all jobs
print('[INFO] setting up batch jobs.')

#for subject in subject_ids[:2]:    
    #by_file_slurm(subject)
    
print('[INFO] submitted batch jobs.')

    

[INFO] setting up batch jobs.
[INFO] submitted batch jobs.


In [28]:
!squeue -u nbytes

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
           3025389     batch betaseri   nbytes PD       0:00      1 (Nodes required for job are DOWN, DRAINED or reserved for jobs in higher priority partitions)
           3025390     batch betaseri   nbytes PD       0:00      1 (Nodes required for job are DOWN, DRAINED or reserved for jobs in higher priority partitions)
           3025919     batch betaseri   nbytes PD       0:00      1 (Nodes required for job are DOWN, DRAINED or reserved for jobs in higher priority partitions)
           3025920     batch betaseri   nbytes PD       0:00      1 (Nodes required for job are DOWN, DRAINED or reserved for jobs in higher priority partitions)
           3025921     batch betaseri   nbytes PD       0:00      1 (Nodes required for job are DOWN, DRAINED or reserved for jobs in higher priority partitions)
           3025922     batch betaseri   nbytes PD       0:00      1 (Nodes required for job are DOWN, DRA

Look at the input directory:

In [44]:

!ls '/projects/niblab/experiments/bbx/data/bids/derivatives/preprocessed/sub-001/ses-1/analysis'

analysis  anat	func
beta


In [47]:
!ls /projects/niblab/parcellations/bigbrain300

bad	     WashU_300rois_MNI152_3mm_origOrder.nii
MNI_3mm_sep  WashU_300rois_order.txt


## Setup Feat files

Confound files:

In [13]:
#!ls '/projects/niblab/experiments/bbx/data/bids/derivatives/preprocessed/sub-001/ses-1/func/motion_assessment'



## Run Slurm jobs

## Concatenate PEs

In [ ]:
subject_path='/projects/niblab/experiments/bbx/data/bids/derivatives/preprocessed/sub-*'
subject_folders=sorted(glob.glob(subject_path))

data_dict={}
bad_subs=[]
